In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
import sys    
path_to_module = '/content/gdrive/MyDrive/TIP-IM'
sys.path.append(path_to_module)

Mounted at /content/gdrive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip3 install numpy==1.16.2
!pip3 install scipy==1.2.1

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import random
import torch
import os
import sys
sys.path.append('align_methods')
from align import align, re_align
from utils import save_priv, cosdistance, inference
import math
import argparse
from scipy.misc import imread
from collections import OrderedDict
from get_model import getmodel
from tqdm import tqdm
from input_diversify import input_diversity
import cv2
from config import threshold
from torch.autograd import Variable
from mmd import mmd_loss







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [ ]:
#parser.add_argument('--input_file', help='Input directory with images.', type=str, default='data/pairs.txt')

#parser.add_argument('--num_iter', help='Number of iterations.', type=int, default=100)

#parser.add_argument('--src_model', help='White-box model', type=str, default='ArcFace', choices=threshold.keys())

#parser.add_argument('--ensemble_image', help='whether to use multiple images', type=int, default=0, choices=[0, 1])

#parser.add_argument('--batch_size', help='batch size', type=int, default=1)

#parser.add_argument('--output', help='output directory', type=str, default='output/exppriv')

#parser.add_argument('--gain', help='gain function', type=str, default='gain3')

#parser.add_argument('--target_nums', help='target_nums', type=int, default=10)

#parser.add_argument('--gamma', help='gamma', type=float, default=0.0)

#parser.add_argument('--norm', help='select norm', type=str, default='l2')
seed = 0

In [ ]:
def main():
    model, img_shape = getmodel('ArcFace') # White-box Model
    iters = 100
    norm = 'l2'
    gamma = 0.0

    cnt_pairs = 0
    candidate_e = [12]
    func = {
        'gain1': Gain1,
        'gain2': Gain2,
        'gain3': Gain3
        }
    
    # conduct attacker
    attacker_persons = []
    with open('/content/gdrive/MyDrive/TIP-IM/data/pairs.txt', 'r') as f:
        for pair in f.readlines()[:500]:
            attacker_person = pair.strip()
            attacker_persons.append(attacker_person)
            
    random.seed(seed)
    cnt = 0
    batch_size = 1

    # conduct targets, only 10 identities!
    target_nums = 10
    aligned_target_imgs = []
    target_persons = []
    with open('/content/gdrive/MyDrive/TIP-IM/targets/target_attacks.txt', 'r') as f:
        for line in f.readlines():
            target_person = line.strip()
            target_persons.append(target_person)

    for i in range(target_nums):
        target_img = imread(target_persons[i]).astype(np.float32)
        align_target_img, _= align(target_img)
        align_target_img = cv2.resize(align_target_img, (img_shape[1], img_shape[0]))
        aligned_target_imgs.append(align_target_img)
    aligned_target_imgs = np.array(aligned_target_imgs)        

    # extract features of targets
    target_feas = inference(aligned_target_imgs, model, image_shape = img_shape)
    target_feas = Variable(target_feas)
    
    # extract initial features of attackers
    n = len(attacker_persons)
    m = 1.0
    for i in tqdm(range(0, n, batch_size)):
        l, r = i, min(i + batch_size, n)
        original_images = []
        aligned_images = []
        aligned_names = []
        M = []
        for j in range(l, r):
            attack_name, id1, id2 = attacker_persons[j].split('\t')
            original_img = imread("/content/gdrive/MyDrive/TIP-IM/data/LFW/{}/{}_{:04d}.jpg".format(attack_name, attack_name, int(id2))).astype(np.float32)
            original_images.append(original_img)
            align_img, tmpM = align(original_img)
            align_img = cv2.resize(align_img, (img_shape[1], img_shape[0]))
            aligned_images.append(align_img)
            aligned_names.append("{}_{:04d}".format(attack_name, int(id2)))
            M.append(tmpM)
        aligned_images = np.array(aligned_images)
        input_init = torch.Tensor(aligned_images).cuda()
        input_init = input_init.permute(0, 3, 1, 2)
        init_feas = model.forward(input_init)
        init_feas = Variable(init_feas)

        # craft protected images 
        for epsilon in candidate_e:
            alpha = 1.5 * epsilon / iters
            inputs = torch.Tensor(aligned_images.copy()).cuda()
            inputs = inputs.permute(0, 3, 1, 2)
            sum_grad = torch.zeros_like(inputs)
            min_img = torch.clamp(inputs - epsilon, min=0)
            max_img = torch.clamp(inputs + epsilon, max=255)
            adv_images = inputs.detach().clone().requires_grad_(True)
            for iii in range(iters):
                std_proj = random.uniform(0.01, 0.1)
                std_rotate = random.uniform(0.01, 0.1)
                tmp_advs = []
                tmp_grads = []
                tmp_losses = []
                model.zero_grad()
                # introduce input diversity for generalizaion
                images = input_diversity(adv_images, std_proj, std_rotate)
                adv_feas = model.forward(images)

                # compute image-level natural loss by MMD. 
                loss_mmd = mmd_loss(adv_images.clone().reshape(batch_size,-1), 
                        inputs.clone().reshape(batch_size,-1))
                
                # search optimal target 
                for idx in range(target_nums):
                    model.zero_grad()
                    loss_i = torch.mean((adv_feas - init_feas) ** 2)
                    loss_t = torch.mean((adv_feas - target_feas[idx]) ** 2)

                    # total loss 
                    '''
                    loss_mmd does not affect search direction of targets set, just for updating images.
                    '''
                    loss = loss_t - loss_i + gamma * loss_mmd
   
                    loss.backward(retain_graph = True)
                    grad = adv_images.grad.data.clone()
                    grad = grad / grad.abs().mean(dim=[1, 2, 3], keepdim=True)
                    
                    tmp_sum_grad = m * sum_grad.clone() + grad
                    adv_images.grad.data.zero_()
                    tmp_adv_images = adv_images.data.clone()
                    # infty norm
                    if norm == 'linf':
                        tmp_adv_images = tmp_adv_images - torch.sign(tmp_sum_grad) * alpha
                    elif norm == 'l2':
                        factor = np.sqrt(np.prod(img_shape)* 3)
                        grad2d = tmp_sum_grad.reshape((tmp_sum_grad.size(0), -1))
                        gradnorm = grad2d.norm(p=2, dim=1, keepdim=True)
                        grad_unit = grad2d / gradnorm
                        delta = -torch.reshape(grad_unit, tmp_sum_grad.size()) * alpha * factor
                        tmp_adv_images = tmp_adv_images + delta
                    
                    tmp_adv_images = torch.max(tmp_adv_images, min_img)
                    tmp_adv_images = torch.min(tmp_adv_images, max_img)
                    tmp_grads.append(tmp_sum_grad)
                    tmp_advs.append(tmp_adv_images)
                    tmp_losses.append(loss.data.unsqueeze(0))
               
                tmp_losses = torch.cat(tmp_losses)
                # find best index
                best_idx = submodular(target_feas, init_feas, tmp_advs, model, func['gain3'])
                sum_grad = tmp_grads[best_idx]
                
                adv_images = tmp_advs[best_idx]
                #print(tmp_losses[best_idx], best_idx)
                adv_images = adv_images.detach().requires_grad_(True)
            adv_images = adv_images.detach().permute(0, 2, 3, 1).cpu().numpy()
                
            for j in range(batch_size):
                attacker_name = attacker_persons[l + j].split('\t')[0]
                _ = save_priv(adv_images[j], aligned_images[j], original_images[j], attacker_name, M[j], epsilon, 'ArcFace', 'output/exppriv')

def submodular(target_feas, init_feas, tmp_advs, model, Gain):
    tmp_advs = torch.cat(tmp_advs)
    
    gains = torch.zeros(size = (len(target_feas),), dtype = torch.float32)
    tmpadv_feas = model.forward(tmp_advs)
    for idx in range(len(tmpadv_feas)):
        gains[idx] = Gain(tmpadv_feas[idx].unsqueeze(0), init_feas, target_feas, idx)
    best_idx = torch.argmax(gains)
    return best_idx

def L2distance(x, y):
	return torch.sqrt(torch.sum((x - y)**2, dim = 1))

def Gain1(adv_fea, init_feas, target_feas, idx=0):
    d1 = L2distance(adv_fea, init_feas)
    d2 = torch.sum(torch.exp(d1 - L2distance(adv_fea, target_feas)))
    return torch.log(1.0 + d2)

def Gain2(adv_fea, init_feas, target_feas, idx=0):
    d1 = L2distance(adv_fea, init_feas)
    d2 = torch.min(torch.exp(d1 - L2distance(adv_fea, target_feas)))
    return torch.log(1.0 + d2)

def Gain3(adv_fea, init_feas, target_feas, idx=0):
    d1 = L2distance(adv_fea, init_feas)
    d2 = torch.max(torch.exp(d1 - L2distance(adv_fea, target_feas)))
    return torch.log(1.0 + d2)


if __name__ == '__main__':
    main()

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout, MaxPool2d, \
    AdaptiveAvgPool2d, Sequential, Module
from collections import namedtuple
from FaceModel import FaceModel


# Support: ['IR_50', 'IR_101', 'IR_152', 'IR_SE_50', 'IR_SE_101', 'IR_SE_152']


class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


def l2_norm(input, axis=1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output


class SEModule(Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(
            channels, channels // reduction, kernel_size=1, padding=0, bias=False)

        nn.init.xavier_uniform_(self.fc1.weight.data)

        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(
            channels // reduction, channels, kernel_size=1, padding=0, bias=False)

        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return module_input * x


class bottleneck_IR(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False), BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False), PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False), BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class bottleneck_IR_SE(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR_SE, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False),
            PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False),
            BatchNorm2d(depth),
            SEModule(depth, 16)
        )

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''


def get_block(in_channel, depth, num_units, stride=2):

    return [Bottleneck(in_channel, depth, stride)] + [Bottleneck(depth, depth, 1) for i in range(num_units - 1)]


def get_blocks(num_layers):
    if num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=8),
            get_block(in_channel=128, depth=256, num_units=36),
            get_block(in_channel=256, depth=512, num_units=3)
        ]

    return blocks


class Backbone(Module):
    def __init__(self, input_size, num_layers, mode='ir'):
        super(Backbone, self).__init__()
        assert input_size[0] in [112, 224], "input_size should be [112, 112] or [224, 224]"
        assert num_layers in [50, 100, 152], "num_layers should be 50, 100 or 152"
        assert mode in ['ir', 'ir_se'], "mode should be ir or ir_se"
        blocks = get_blocks(num_layers)
        if mode == 'ir':
            unit_module = bottleneck_IR
        elif mode == 'ir_se':
            unit_module = bottleneck_IR_SE
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1, bias=False),
                                      BatchNorm2d(64),
                                      PReLU(64))
        if input_size[0] == 112:
            self.output_layer = Sequential(BatchNorm2d(512),
                                           Dropout(),
                                           Flatten(),
                                           Linear(512 * 7 * 7, 512),
                                           BatchNorm1d(512))
        else:
            self.output_layer = Sequential(BatchNorm2d(512),
                                           Dropout(),
                                           Flatten(),
                                           Linear(512 * 14 * 14, 512),
                                           BatchNorm1d(512))

        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel,
                                bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)

        self._initialize_weights()

    def forward(self, x):
        x = (x - 127.5) / 128
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()


def IR_50(input_size):
    """Constructs a ir-50 model.
    """
    model = Backbone(input_size, 50, 'ir_se')

    return model

class ArcFace(FaceModel):
    def __init__(self, **kwargs):
        net = IR_50((112, 112))
        url = 'http://ml.cs.tsinghua.edu.cn/~xiaoyang/face_models/ArcFace/model_ir_se50.pth'
        channel = 'rgb'
        FaceModel.__init__(
            self,
            net=net,
            url=url,
            channel=channel,
            **kwargs)



In [ ]:
from six.moves import urllib
url = 'http://ml.cs.tsinghua.edu.cn/~xiaoyang/face_models/ArcFace/model_ir_se50.pth'
model_name = url.split('/')[-1]
net = IR_50((112, 112))

if not os.path.exists('./ckpts/'):
	try:
		os.makedirs('./ckpts/')
	except OSError as e:
		if e.errno != errno.EEXIST:
			raise
urllib.request.urlretrieve(url,'./ckpts/{}'.format(model_name))
print('Finish downloading')
print('Load checkpoint')
checkpoint = torch.load('./ckpts/{}'.format(model_name), map_location=lambda storage, loc: storage.cuda())

if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
	net.load_state_dict(checkpoint['state_dict'])
else:
	net.load_state_dict(checkpoint)

Finish downloading
Load checkpoint


##get name list

In [5]:
attacker_persons = []
with open('/content/gdrive/MyDrive/TIP-IM/pairs.txt', 'r') as f:
    for pair in f.readlines()[:87]:
        attacker_person = pair.strip()
        attacker_persons.append(attacker_person)


# attack_name, id1, id2 = attacker_persons[j].split('\t')
# original_img = imread("/content/gdrive/MyDrive/TIP-IM/data/LFW/{}/{}_{:04d}.jpg".format(attack_name, attack_name, int(id2))).astype(np.float32)
model, img_shape = getmodel('ArcFace') # White-box Model
batch_size = 1
n = len(attacker_persons)
m = 1.0
names = []
for i in tqdm(range(0, n, batch_size)):
    l, r = i, min(i + batch_size, n)
    
    for j in range(l, r):
        attack_name, id1, id2 = attacker_persons[j].split('\t')
        names.append(attack_name)
        
        
    
names = set(names)
names = list(names)
print(len(names))

Load existing checkpoint
No existing checkpoint, now downloading online
Finish downloading
Load checkpoint


100%|██████████| 87/87 [00:00<00:00, 423323.03it/s]

50


##gallery set

In [17]:
gallery_set = []
gallery = os.listdir('/content/gdrive/MyDrive/TIP-IM/gallery')
for image in os.listdir('/content/gdrive/MyDrive/TIP-IM/gallery'):
  gallery_img = imread('/content/gdrive/MyDrive/TIP-IM/gallery/'+image).astype(np.float32)
  align_target_img, _= align(gallery_img)
  align_target_img = cv2.resize(align_target_img, (img_shape[1], img_shape[0]))
  gallery_feas = inference(align_target_img, model, image_shape = img_shape)
  gallery_feas = Variable(gallery_feas)
  gallery_feas = gallery_feas.cpu().data
  gallery_feas = np.array(gallery_feas)
  gallery_set.append(gallery_feas)
gallery_set = np.array(gallery_set).reshape(233,-1)
print(gallery_set.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


(233, 512)


##Protected image

In [7]:
protected_set = []
for name in names:
  protected_img = imread('/content/gdrive/MyDrive/TIP-IM/exppriv/'+name+'-12.png').astype(np.float32)
  protected_feas = inference(protected_img, model, image_shape = img_shape)
  protected_feas = Variable(protected_feas)
  protected_feas = protected_feas.cpu().data
  protected_feas = np.array(protected_feas)
  protected_set.append(protected_feas)
# protected_set = np.array(protected_set)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


##compare

In [15]:
top5 = []
for pro_img in protected_set:
  distance = []
  min = []
  for gal_img in gallery_set:
    dist = np.linalg.norm(pro_img - gal_img)
    distance.append(dist)

  for i in range(5):
    
    ind = np.argmin(distance)
    min.append(ind)
    distance[ind] = 9999
  top5.append(np.array(min))
print(top5)

[array([  7,  51, 131, 198,  50]), array([210, 120, 115, 173,  98]), array([169,  64,  95,  16,  34]), array([ 16, 202,  34, 169, 203]), array([212,  25, 170, 108, 102]), array([220, 128, 211,  42,  25]), array([ 16, 202, 182, 228,  23]), array([212,  25,  96, 102, 109]), array([ 16, 202, 132,  22,  34]), array([216, 215, 147,  37, 156]), array([  7, 131,  51, 178, 181]), array([220, 128, 166, 111, 108]), array([169,  64,  46,  86,  34]), array([212,  25,  28, 108, 131]), array([  7,  51, 131, 198,   5]), array([212,  25, 102,  80, 227]), array([212,  25, 220,  24,   2]), array([ 49,  58, 115, 121,  98]), array([ 16, 202, 169,  71,  42]), array([ 31, 208, 120, 185,  54]), array([ 49,  58, 191, 190,  45]), array([216, 215,  37, 152, 158]), array([ 16, 202,  34, 198,  51]), array([212,  25, 227, 192,  35]), array([ 16, 202, 228, 167,  23]), array([169,  64, 159, 179, 177]), array([182, 103,  16, 130,  47]), array([49, 58, 82, 81, 39]), array([220, 128,   2,  12, 111]), array([ 49,  58,  

In [22]:
#taeget set
target_nums = 10
aligned_target_imgs = []
target_persons = []
target_names = []
with open('/content/gdrive/MyDrive/TIP-IM/target_attacks.txt', 'r') as f:
    for line in f.readlines():
        target_person = line.strip()
        # name, id1, id2 = target_person.split('\t')
        # target_names.append(name)
        target_persons.append(target_person)
        temp = target_person.split('/')
        target_names.append(temp[-2])

for i in range(target_nums):
    target_img = imread(target_persons[i]).astype(np.float32)
    align_target_img, _= align(target_img)
    align_target_img = cv2.resize(align_target_img, (img_shape[1], img_shape[0]))
    aligned_target_imgs.append(align_target_img)
aligned_target_imgs = np.array(aligned_target_imgs)        

# extract features of targets
target_feas = inference(aligned_target_imgs, model, image_shape = img_shape)
target_feas = Variable(target_feas)

#name of top5
top5_name = []
for i in top5:
  name_list = []
  for j in i:
    img = gallery[j]
    people_name, num = img.rsplit('_',1)
    name_list.append(people_name)
  top5_name.append(name_list)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()


1.0


##Rank-5-T

In [26]:
#rank-n-t
count = 0
for i in top5_name:
  for j in i:
    if j in target_names:
      count = count + 1
      break
print(count/50)

1.0


##Rank-5-UT

In [25]:
count = 0
for i in range(50):
  if names[i] not in top5_name[i]:
    count = count + 1
print(count/50)

0.82


##Rank-1-T

In [28]:
count = 0
for i in top5_name:
  if i[0] in target_names:
    count = count + 1
print(count/50)

0.98


##Rank-1-UT

In [29]:
count = 0
for i in range(50):
  if names[i] != top5_name[i][0]:
    count = count + 1
print(count/50)

0.98
